In [2]:
import pandas as pd
import numpy as np

# Random dataset

In [3]:
def random_bool(size):
    return np.random.rand(size) > 0.5

def random_int(min, max, size):
    return np.random.randint(min, max, size=size)

def random_float(min, max, size):
    return np.random.uniform(min, max, size=size)


def index_id(max):
    list = []
    for i in range(1, max):
        list.append('ID' + str(i))
    return list

In [4]:
pre_op = pd.DataFrame({'IDno': index_id(501), 'Død inden for 1 år af operation': random_bool(500), 'age': random_int(18, 89, 500), 'sex': random_bool(500), 'Respiratory Disease': random_bool(500),
       'Circulatory Organ Disease': random_bool(500), 'Type 1 Diabetes': random_bool(500), 'Type 2 Diabetes': random_bool(500),
       'Other metabolic diseases': random_bool(500), 'Other operation': random_bool(500), 'Genital or urine related diseases': random_bool(500), 'Vægt': random_float(35, 190,500), 'KFN': random_bool(500), 'KFX': random_bool(500), 'KFK': random_bool(500), 'KFM': random_bool(500),
       'KFF': random_bool(500), 'KFP': random_bool(500), 'KFC': random_bool(500), 'KFW': random_bool(500), 'KFJ': random_bool(500), 'Hæmoglobin': random_float(4,12,500), 'Leukocytter': random_float(1, 125, 500),
       'Trombocytter': random_float(20, 1000, 500)})
pre_op = pre_op.replace(True, 1).replace(False, 0)

# Predicting Mortality with pre_op data

## LR

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

#TODO other params?
def lr_pred(X_train, y_train, X_test):
    lr = LogisticRegression()
    lr_grid = {'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
            'C': [100, 10, 1.0, 0.1, 0.01],
                'max_iter': 200}
    
    grid_result = GridSearchCV(estimator = lr, param_grid = lr_grid, scoring='precision', cv=5)
    grid_result.fit(X_train,y_train)
    
    best_model = grid_result.best_estimator_ 

    lr_preds = best_model.predict_proba(X_test)
    return lr_preds

    #TODO Take Best result of the fit or can i just return the predict 

## Random forrest

In [19]:
from sklearn.ensemble import RandomForestClassifier

def rf_pred(X_train, y_train, X_test):
    rf = RandomForestClassifier()

    rf_grid = {'max_depth': [80, 90, 100, 110],
    'max_features': [2,5,10,15,22],
    'n_estimators': [100, 200, 300, 1000]}
    
    grid_result = GridSearchCV(estimator = rf, param_grid = rf_grid, scoring='precision', cv=5)
    grid_result.fit(X_train,y_train)
    
    best_model = grid_result.best_estimator_ 

    rf_preds = best_model.predict_proba(X_test)
    return rf_preds
    

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,_ = train_test_split(X,y, test_size=0.3)
rf_pred(X_train,y_train,X_test)

array([[0.17 , 0.83 ],
       [0.595, 0.405],
       [0.44 , 0.56 ],
       [0.75 , 0.25 ],
       [0.53 , 0.47 ],
       [0.65 , 0.35 ],
       [0.545, 0.455],
       [0.385, 0.615],
       [0.435, 0.565],
       [0.75 , 0.25 ],
       [0.605, 0.395],
       [0.215, 0.785],
       [0.455, 0.545],
       [0.67 , 0.33 ],
       [0.63 , 0.37 ],
       [0.27 , 0.73 ],
       [0.585, 0.415],
       [0.52 , 0.48 ],
       [0.5  , 0.5  ],
       [0.4  , 0.6  ],
       [0.205, 0.795],
       [0.76 , 0.24 ],
       [0.32 , 0.68 ],
       [0.295, 0.705],
       [0.24 , 0.76 ],
       [0.41 , 0.59 ],
       [0.44 , 0.56 ],
       [0.295, 0.705],
       [0.315, 0.685],
       [0.145, 0.855],
       [0.455, 0.545],
       [0.31 , 0.69 ],
       [0.62 , 0.38 ],
       [0.48 , 0.52 ],
       [0.645, 0.355],
       [0.82 , 0.18 ],
       [0.51 , 0.49 ],
       [0.255, 0.745],
       [0.32 , 0.68 ],
       [0.46 , 0.54 ],
       [0.495, 0.505],
       [0.355, 0.645],
       [0.58 , 0.42 ],
       [0.5

## Gradient boost


## MLP

# Grouped cross-validation

In [5]:
from sklearn.model_selection import GroupKFold

X = pre_op.loc[:, pre_op.columns != 'Død inden for 1 år af operation']
X = X.loc[:, X.columns != 'IDno']
y = pre_op['Død inden for 1 år af operation']

outer_cv = GroupKFold(n_splits = 5)
inner_cv = GroupKFold(n_splits = 5)

for outer_train_idx, outer_test_idx in outer_cv.split(X, y, groups = pre_op['IDno']):
    X_outer_train, y_outer_train = X.iloc[outer_train_idx], y.iloc[outer_train_idx]
    X_outer_test, y_outer_test = X.iloc[outer_test_idx], y.iloc[outer_test_idx]

    for inner_train_idx, inner_test_idx in inner_cv.split(X_outer_train, y_outer_train, groups=pre_op['IDno'].iloc[outer_train_idx]):
        X_inner_train, y_inner_train = X_outer_train.iloc[inner_train_idx], y_outer_train.iloc[inner_train_idx]
        X_inner_test, y_inner_test = X_outer_train.iloc[inner_test_idx], y_outer_train.iloc[inner_test_idx]

        #TODO Make the perfomance class that returns PR, AUC, ROC and PRC